# Recommender System with GraphLab
In which we demonstrate some features of GraphLab while building a movie recommendation system

In [ ]:
import graphlab as gl



The factorization recommender tries to minimize the function:
\begin{equation}
\min_{\mathbf{w},\mathbf{a},\mathbf{b},\mathbf{U},\mathbf{V}} \frac{1}{\lvert\mathcal{D}\rvert} \sum_{i,j,r_{i,j} \in \mathcal{D}}{ \mathcal{L}\bigl(score\left(i,j\right),r_{i,j} \bigr) + \lambda_{1}\left(\lVert\mathbf{w}\rVert_{2}^{2} + \Vert\mathbf{a}\Vert_{2}^{2} + \Vert\mathbf{b}\Vert_{2}^{2}\right) + \lambda_{2}\left(\Vert\mathbf{U}\Vert_{2}^{2} + \Vert\mathbf{V}\Vert_{2}^{2}\right)} 
\end{equation}
where:
\begin{equation}
score\left(i,j\right) = \mu + w_i + w_j +\mathbf{a}^{T}\mathbf{x}_{i} + \mathbf{b}^{T}\mathbf{y}_{i} + \mathbf{u}_{i}^{T}\mathbf{v}_{j}
\end{equation}
and
$\mu$ is the overall average rating, $w_i$ is the user bias, $w_j$ is the item bias, $\mathbf{a}$ and $\mathbf{x}$ are the user data, $\mathbf{a}$ and $\mathbf{x}$ are the item data, and $\mathbf{u}$ and $\mathbf{v}$ are the user and item factors

We do a lot with a recommender model. First we can get recommendations for a user:
```python
top_movies = recommender.recommend([1])['movieid']
movies.filter_by(top_movies, 'movieid')
```

We can find similar items to a movie:
```python
inception_id = movies.filter_by('Inception (2010)', 'title')['movieid']
similar_movies = recommender.get_similar_items(inception_id)['similar']
movies.filter_by(similar_movies, 'movieid')
```
